In [93]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

Input Problem parameters

In [94]:
import json

l=["small", "medium", "large"]
res = dict()

for taille in l:
    nom_fichier = taille+".json"
    fichier = open(nom_fichier)
    print("I opened ", nom_fichier, " filed. \n")

    data = json.load(fichier)
    print("I loaded ", nom_fichier, " data in a dict. \n")

    res[taille] = data
    print("I stored ", nom_fichier, " data in res. \n")

    fichier.close()

I opened  small.json  filed. 

I loaded  small.json  data in a dict. 

I stored  small.json  data in res. 

I opened  medium.json  filed. 

I loaded  medium.json  data in a dict. 

I stored  medium.json  data in res. 

I opened  large.json  filed. 

I loaded  large.json  data in a dict. 

I stored  large.json  data in res. 



In [128]:
## à changer selon le fichier que l'on veut
## traiter
## l = ["small", "medium", "large"]


taille = "small"

In [129]:
horizon = res[taille]["horizon"]
qualifications = res[taille]["qualifications"]
staff = res[taille]["staff"]
jobs = res[taille]["jobs"]

In [130]:
noms_employes = []
for person in staff:
    noms_employes.append(person["name"])

noms_projet = []
for projet in jobs:
    noms_projet.append(projet["name"])

In [131]:
nombre_projets = len(noms_projet)
nombre_qualif = len(qualifications)
nombre_employes = len(noms_employes)

Définissons les variables

In [132]:
## besoin_i_j, le projet j a besoin de la qualif i
besoin = [[0]*nombre_projets for _ in range(nombre_qualif)]

## conge_k_n, l'employé k est en congés au jours n (1 s'il travaille et 0 s'il est en congés)
conge = [[1]*horizon for _ in range(nombre_employes)]

## qualif_i_k, l'employé k a la qualif i
qualif = [[0]*nombre_employes for _ in range(nombre_qualif)]

## d_j, date de livraison selon les projets
d = [0]*nombre_projets

## p_j, pénalités selon les projets
p = [0]*nombre_projets

## b_j, bénéfices selon les projets
b = [0]*nombre_projets



##########################################
##########################################



for job in jobs:
    ## jobs is list of dict
    ## job is dict

    num_projet = noms_projet.index(job["name"])
    b[num_projet] = job["gain"]
    d[num_projet] = job["due_date"]
    p[num_projet] = job["daily_penalty"]

    for qualification, jours in job["working_days_per_qualification"].items():
        num_qualif = qualifications.index(qualification)
        besoin[num_qualif][num_projet] = jours
    

for person in staff:
    num_person = noms_employes.index(person["name"])

    for qual in person["qualifications"]:    
        num_qualif = qualifications.index(qual)
        qualif[num_qualif][num_person] = 1
    
    for date in person["vacations"]:
        if date is not None:
            conge[num_person][date] = 0

{'name': 'Olivia', 'qualifications': ['B', 'A', 'C'], 'vacations': [1, 2, 3, 4, 5]}
8
3
4
{'name': 'Liam', 'qualifications': ['E', 'A', 'D'], 'vacations': [6, 7, 8, 9, 10]}
0
3
7
{'name': 'Emma', 'qualifications': ['B', 'H'], 'vacations': [21, 22, 23, 24, 25]}
8
5
{'name': 'Noah', 'qualifications': ['C', 'H', 'G', 'D'], 'vacations': [26, 27, 28, 29, 30]}
4
5
6
7
{'name': 'Amelia', 'qualifications': ['E', 'I', 'G', 'J', 'F'], 'vacations': []}
0
1
6
2
9
{'name': 'Oliver', 'qualifications': ['J', 'F', 'G'], 'vacations': [32, 33, 31]}
2
9
6


In [133]:
print("besoin : ", besoin)
print("qualif : ", qualif)
print("conge : ", conge)
print("d : ", d)
print("p : ", p)
print("b : ", b)

besoin :  [[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 5, 5, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 4, 0, 0, 5, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 6, 0, 3, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 4, 0, 8, 10], [4, 4, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 4, 0, 0, 5, 7, 4, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 7, 0, 0, 8, 0, 0, 0, 0, 0, 5, 0, 2, 0, 1, 4, 0, 0, 6, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 5, 0, 0, 3, 0, 2, 4, 4, 4, 0, 0, 0], [0, 4, 0, 0, 8, 0, 0, 5, 0, 0, 0, 4, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4, 2, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 6, 0, 6, 6, 0, 0, 0, 0, 0, 0]]
qualif :  [[0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1], [0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1]]
conge :

Définissons les paramètres

In [134]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon for _ in range(nombre_employes)] for _ in range(nombre_projets)] for _ in range(nombre_qualif)]

## begin_j, la date de debut effective du projet j
begin = [0]*nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end = [0]*nombre_projets